In [1]:
import os,sys #user to create and modify file name and save it
from netCDF4 import Dataset#to read the .nc files
import pandas as pd #for dataframe activities
from datetime import datetime,timedelta #for the time related activities
import matplotlib.pyplot as plt #for the Ploting purpose
import numpy as np #numerical fucntions
import datetime 
import warnings #to ignore unnecessary warnings
import xarray as xr
import numpy as np
import regionmask
import geopandas as gp
import glob                                                                 
from pyhdf.SD import SD, SDC
warnings.filterwarnings('ignore')

# Correction and clipping of RH

In [4]:
os.chdir('/Volumes/PtatoBasket/ACIML/Datasets/ERA-5 RH/Winter')
ds=xr.open_dataset('RH-MAM_daily.nc')


### Reading out the dates

In [5]:
from datetime import datetime,timedelta,date
RH=ds.r
time,pressure,lat,lon=RH.indexes.values()
x=list(time)
y=[]

for n in x:
    y.append(n.date())

### Getting Out the dates in the format spec

In [7]:
import os
from datetime import datetime,timedelta,date
date_list=[]
for yr in range(2005,2022):
    month=[1,2,12]
    for mon in month:
        for day in range(1,32):
            try:
                d=date(yr,mon,day)
                delta=timedelta(days=1)
                year=d.year
                doy=d.timetuple().tm_yday
                doy="{:03d}".format(doy)
                # date_list.append('A'+str(year)+str(doy)+'.hdf')#MODIS-GESDISC
                # date_list.append(d.strftime('%Y%m%d'))#IMGERG/PBL
                date_list.append(d)#IMGERG/PBLH
                d+=delta
            except ValueError:
                pass
              

### Getting the list of unwanted dates

In [8]:
  
missing_dates=list(set(y)-set(date_list))
indexes = [y.index(x) for x in missing_dates]
unwanted_times=[time[x] for x in indexes]


### Taking out the unwanted dates from the original file

In [9]:

for x in unwanted_times:
    # print(x)
    RH=RH.isel(time=RH['time']!=x)
    
RH=RH.mean(dim = 'level')

## Clipping and Saving

In [10]:

#Clipping to shape File and Creating Mask File
shapefile ="/Volumes/ACIML/Main/SHPs/BOB_OCEAN/BOB_OCEAN.shp"
countries=gp.read_file(shapefile,engine='pyogrio')
c_list=list(countries['featurecla'])
c_list_unique=set(list(countries['featurecla']))
indexes=[c_list.index(x) for x in c_list_unique]
countries_mask_poly=regionmask.Regions(outlines=countries.geometry[indexes],name='featurecla',numbers=indexes,names=countries.featurecla[indexes])
print('It is Clipping')
mask=countries_mask_poly.mask(RH,lat_name='latitude',lon_name='longitude')
# mask.to_netcdf('BOB_MASK.nc')

# mask=xr.open_dataarray('/Volumes/ACIML/Main/SHPs/BOB_MASK.nc')
masked_shape=RH.where(mask==0)
min_lon = 78.00
max_lon = 96.00
min_lat = 8.00
max_lat = 23.00

cropped_ds = masked_shape.sel(latitude=slice(max_lat,min_lat), longitude=slice(min_lon,max_lon))

cropped_ds.to_netcdf('WinterDaily_RHavg.nc')

It is Clipping


# For IMGERG Precipitation

In [ ]:
st=time.time()
os.chdir('/Volumes/PtatoBasket/Dataset_New/IMERG')
xxx=[]
a=[]
for file in list(glob.glob('PreMon/*.nc4')):

    ds=xr.open_dataset(file)

    # Retrieve attributes.
    attrs = ds.attrs
    dat=attrs["BeginDate"]
    dat=datetime.strptime(dat,'%Y-%m-%d')
    # January 1st, 1900
    start_datetime = datetime(1900, 1, 1)

    # Calculate the time delta (difference) in hours
    delta = (dat - start_datetime).total_seconds() / 3600


    xxx.append(delta)


In [7]:
from datetime import datetime,timedelta
arr=str('A2005001.hdf').split('.')
year=str(arr[0])[1:5]
days=str(arr[0])[5:]
dat=datetime(int(year), 1, 1) + timedelta(int(days) - 1)
# dat=(dat-datetime(2005,1,1)).days
# January 1st, 1900
start_datetime = datetime(1900, 1, 1)

# Calculate the time delta (difference) in hours
delta = (dat - start_datetime).total_seconds() / 3600


# For PBLH

In [4]:
os.chdir('/Volumes/PtatoBasket/ACIML/Datasets/PBLH/')
ds=xr.open_dataset('AWinterDaily_PBLH.nc')

#Clipping to shape File and Creating Mask File
shapefile ="/Volumes/ACIML/Main/SHPs/BOB_OCEAN/BOB_OCEAN.shp"
countries=gp.read_file(shapefile,engine='pyogrio')
c_list=list(countries['featurecla'])
c_list_unique=set(list(countries['featurecla']))
indexes=[c_list.index(x) for x in c_list_unique]
countries_mask_poly=regionmask.Regions(outlines=countries.geometry[indexes],name='featurecla',numbers=indexes,names=countries.featurecla[indexes])
print('It is Clipping')
mask=countries_mask_poly.mask(ds,lat_name='lat',lon_name='lon')

masked_shape=ds.TCZPBL.where(mask==0)

min_lon = 78.00
max_lon = 96.00
min_lat = 8.00
max_lat = 23.00

cropped_ds = masked_shape.sel(lat=slice(min_lat,max_lat), lon=slice(min_lon,max_lon))

cropped_ds.to_netcdf('WinterDaily_PBLH.nc')

It is Clipping


In [3]:
ds

<xarray.Dataset>
Dimensions:    (time: 1534, bnds: 2, lon: 30, lat: 31)
Coordinates:
  * time       (time) datetime64[ns] 2005-01-01T12:00:00 ... 2021-12-31T12:00:00
  * lon        (lon) float64 78.12 78.75 79.38 80.0 ... 94.38 95.0 95.62 96.25
  * lat        (lat) float64 8.0 8.5 9.0 9.5 10.0 ... 21.0 21.5 22.0 22.5 23.0
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) datetime64[ns] ...
    TCZPBL     (time, lat, lon) float32 ...
Attributes: (12/34)
    CDI:                               Climate Data Interface version 2.0.5 (...
    Conventions:                       CF-1
    History:                           Original file generated: Tue Oct 14 23...
    Comment:                           GMAO filename: d5124_m2_jan00.tavg1_2d...
    Filename:                          MERRA2_300.tavg1_2d_flx_Nx.20050101.nc4
    Institution:                       NASA Global Modeling and Assimilation ...
    ...                                ...
    RangeEndingDate:                   2005-01-01
    RangeEndingTime:                   23:59:59.000000
    DODS_EXTRA.Unlimited_Dimension:    time
    history:                           Wed Mar 15 12:53:41 2023: cdo -daymean...
    frequency:                         day
    CDO:                               Climate Data Operators version 2.0.5 (...